In [1]:
%load_ext lab_black
"""Black Formatter for lab"""

import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import IndexSlice as idx
from pathlib import Path
import quantstats as qs
import sqlite3
import yfinance as yf

pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 80)
# pd.set_option("display.precision", 2)
pd.set_option("display.float_format", lambda x: "%.4f" % x)

In [4]:
def get_table(table_name, test_number=None):
    """
    Collects a table for an individual test.
    """
    # Creates connection and downloads the latest database data.
    engine = create_db_connection()

    if test_number:
        sql_v = f"SELECT * FROM {table_name} WHERE test_number='{test_number}';"
    else:
        sql_v = f"SELECT * FROM {table_name};"

    df = pd.read_sql(sql_v, con=engine)

    try:
        df["Date"] = pd.to_datetime(df["Date"])
        df["Date"] = df["Date"].dt.date
    except:
        pass

    try:
        del df["index"]
    except:
        pass

    return df


def create_db_connection():
    """
    Opens a database connection.
    """
    dir = Path("data")
    filename = "results.db"
    filepath = dir / filename
    return sqlite3.connect(filepath)


engine = create_db_connection()
# Get all the table names.
sql = "SELECT name FROM sqlite_master WHERE type IN ('table','view') AND name NOT LIKE 'sqlite_%' ORDER BY 1;"
tables = pd.read_sql(sql, con=engine)["name"].tolist()
tables.remove("transaction")

single_res_tables = ["dimension", "drawdown", "trade_analysis"]
df_combined = pd.DataFrame()
for srt in single_res_tables:
    if srt in tables:
        df = get_table(srt)
        df = df.set_index("test_number")

        if df_combined.empty:
            df_combined = df
        else:
            df_combined = df_combined.join(df)
df_combined.sort_values("pnl_gross_total", ascending=False).head(20)

,batchname,from_date,trade_start,to_date,instrument,benchmark,commission,mult,sma_fast,sma_slow,limit_price,stop_price,trade_size,len,drawdown,moneydown,max_len,max_drawdown,max_moneydown,total_total,total_open,total_closed,streak_won_current,streak_won_longest,streak_lost_current,streak_lost_longest,pnl_gross_total,pnl_gross_average,pnl_net_total,pnl_net_average,won_total,won_pnl_total,won_pnl_average,won_pnl_max,lost_total,lost_pnl_total,lost_pnl_average,lost_pnl_max,long_total,long_pnl_total,...,short_pnl_lost_average,short_pnl_lost_max,short_won,short_lost,len_total,len_average,len_max,len_min,len_won_total,len_won_average,len_won_max,len_won_min,len_lost_total,len_lost_average,len_lost_max,len_lost_min,len_long_total,len_long_average,len_long_max,len_long_min,len_long_won_total,len_long_won_average,len_long_won_max,len_long_won_min,len_long_lost_total,len_long_lost_average,len_long_lost_max,len_long_lost_min,len_short_total,len_short_average,len_short_max,len_short_min,len_short_won_total,len_short_won_average,len_short_won_max,len_short_won_min,len_short_lost_total,len_short_lost_average,len_short_lost_max,len_short_lost_min
test_number,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0b8d79027f,Single Test,2016-01-01,2016-09-01,2020-12-31,FB,SPY,0.0000,1,15,60,0.0900,0.0800,1.0000,0.0000,0.0000,0.0000,52.0000,7.4674,12259.8528,8.0000,0.0000,8.0000,6.0000,6.0000,0.0000,1.0000,67899.5763,8487.4470,67899.5763,8487.4470,7.0000,72904.0279,10414.8611,13619.1846,1.0000,-5004.4516,-5004.4516,-5004.4516,8.0000,67899.5763,...,0.0000,0.0000,0.0000,0.0000,143.0000,17.8750,37.0000,1.0000,142.0000,20.2857,37.0000,7.0000,1.0000,1.0000,1.0000,1.0000,143.0000,17.8750,37.0000,1.0000,142.0000,20.2857,37.0000,7.0000,1.0000,1.0000,1.0000,1.0000,0.0000,0.0000,0.0000,9223372036854775808.0000,0.0000,0.0000,0.0000,9223372036854775808.0000,0.0000,0.0000,0.0000,9223372036854775808.0000
4a221191bc,Single Test,2016-01-01,2016-09-01,2020-12-31,FB,SPY,0.0000,1,30,45,0.0700,0.0500,1.0000,37.0000,4.2770,7121.4544,77.0000,5.9890,8690.5678,10.0000,0.0000,10.0000,0.0000,5.0000,1.0000,1.0000,59382.5745,5938.2574,59382.5745,5938.2574,8.0000,71902.6909,8987.8364,14485.9825,2.0000,-12520.1164,-6260.0582,-7121.4544,10.0000,59382.5745,...,0.0000,0.0000,0.0000,0.0000,149.0000,14.9000,30.0000,1.0000,134.0000,16.7500,30.0000,9.0000,15.0000,7.5000,14.0000,1.0000,149.0000,14.9000,30.0000,1.0000,134.0000,16.7500,30.0000,9.0000,15.0000,7.5000,14.0000,1.0000,0.0000,0.0000,0.0000,9223372036854775808.0000,0.0000,0.0000,0.0000,9223372036854775808.0000,0.0000,0.0000,0.0000,9223372036854775808.0000
3e15e6fd83,Single Test,2016-01-01,2016-09-01,2020-12-31,FB,SPY,0.0000,1,15,120,0.0900,0.0800,1.0000,0.0000,0.0000,0.0000,19.0000,7.3640,9863.8168,5.0000,0.0000,5.0000,5.0000,5.0000,0.0000,0.0000,57865.4998,11573.1000,57865.4998,11573.1000,5.0000,57865.4998,11573.1000,15834.6385,0.0000,0.0000,0.0000,0.0000,5.0000,57865.4998,...,0.0000,0.0000,0.0000,0.0000,189.0000,37.8000,65.0000,8.0000,189.0000,37.8000,65.0000,8.0000,0.0000,0.0000,0.0000,0.0000,189.0000,37.8000,65.0000,8.0000,189.0000,37.8000,65.0000,8.0000,0.0000,0.0000,0.0000,9223372036854775808.0000,0.0000,0.0000,0.0000,9223372036854775808.0000,0.0000,0.0000,0.0000,9223372036854775808.0000,0.0000,0.0000,0.0000,9223372036854775808.0000
bc2ebdb49f,Single Test,2016-01-01,2016-09-01,2020-12-31,FB,SPY,0.0000,1,30,75,0.0900,0.0800,1.0000,0.0000,0.0000,0.0000,18.0000,6.7066,7951.4886,5.0000,0.0000,5.0000,5.0000,5.0000,0.0000,0.0000,56802.8590,11360.5718,56802.8590,11360.5718,5.0000,56802.8590,11360.5718,15890.6493,0.0000,0.0000,0.0000,0.0000,5.0000,56802.8590,...,0.0000,0.0000,0.0000,0.0000,144.0000,28.8000,54.0000,3.0000,144.0000,28.8000,54.0000,3.0000,0.0000,0.0000,0.0000,0.0000,144.0000,28.8000,54.0000,3.0000,144.0000,28.8000,54.0000,3.0000,0.0000,0.0000,0.0000,9223372036854775808.0000,0.0000,0.0000,0.0000,9223372036854775808.0000,0.0000,0.0000,0.0000,9223372036854775808.0000,0.0000,0.0000,0.0000,92

In [3]:
# df_combined.to_excel("multi_test.xlsx")

In [ ]:
print(tables)

### Create a new object for the pairs.

In [ ]:
"""Enter a test number from above."""

tn = "c0149b00ce"

df = get_table("trade_list", tn).sort_values("ref")
df

In [ ]:
df = get_table("value", tn)
df = df.set_index("Date")
bm = yf.download("^GSPC", start=df.index.values[0], end=df.index.values[-1])
vd = df[["Value"]].join(bm["Adj Close"])

# This is the main dataframe with daily data for each of the cient's transactions and values as well
# as the values resulting from the long short pair transaction.
vd.columns = ["value", "bm"]
vd.index = pd.to_datetime(vd.index)

vd["value_return"] = vd["value"].pct_change()
vd["bm_return"] = vd["bm"].pct_change()

# Get log returns
vd["value_log"] = qs.utils.log_returns(vd["value_return"])
vd["bm_log"] = qs.utils.log_returns(vd["bm_return"])

# Rebase to 100
vd["value_rebase"] = qs.utils.rebase(vd["value"])
vd["bm_rebase"] = qs.utils.rebase(vd["bm"])

# Exponentially weighted standard deviation.
vd["value_estdev"] = qs.utils.exponential_stdev(vd["value"])
vd["bm_estdev"] = qs.utils.exponential_stdev(vd["bm"])

# Calculate excess returns over the de.
vd["excess_return"] = qs.utils.to_excess_returns(
    returns=vd["value_return"], rf=vd["bm_return"]
)

# Drawdown details
vd["value_drawdown"] = qs.stats.to_drawdown_series(vd["value"])
vd["bm_drawdown"] = qs.stats.to_drawdown_series(vd["bm"])
vd = vd.fillna(0)
vd.head(40)

In [ ]:
# Drawdown period analysis.
drawdown = qs.stats.drawdown_details(vd["value_drawdown"])
drawdown.head(10)

In [ ]:
# rolling greeks
rolling_greeks = qs.stats.rolling_greeks(
    returns=vd["value_return"], benchmark=vd["bm_return"], periods=252
)
rolling_greeks.tail()

In [ ]:
metrics = qs.reports.metrics(
    returns=vd["value_return"], benchmark=vd["bm_return"], mode="full", display=False
)
metrics.columns = ["value", "bm"]
metrics.head(60)

In [ ]:
# Yearly returns
year_return = pd.concat(
    [
        qs.utils.group_returns(vd["value_return"], vd.index.year),
        qs.utils.group_returns(vd["bm_return"], vd.index.year),
    ],
    axis=1,
)
year_return

In [ ]:
# Yearly/Monthly returns
month_return = pd.concat(
    [
        qs.utils.group_returns(vd["value_return"], [vd.index.year, vd.index.month]),
        qs.utils.group_returns(vd["bm_return"], [vd.index.year, vd.index.month]),
    ],
    axis=1,
)
month_return

In [ ]:
month_return_agg = pd.concat(
    [
        qs.utils.aggregate_returns(vd["value_return"], period="month"),
        qs.utils.aggregate_returns(vd["bm_return"], period="month"),
    ],
    axis=1,
)
month_return_agg

In [ ]:
quarter_return_agg = pd.concat(
    [
        qs.utils.aggregate_returns(vd["value_return"], period="quarter"),
        qs.utils.aggregate_returns(vd["bm_return"], period="quarter"),
    ],
    axis=1,
)
quarter_return_agg

# Plots

In [ ]:
# Daily Returns
qs.plots.daily_returns(returns=vd["value_return"])

In [ ]:
# Drawdown
qs.plots.drawdown(returns=vd["value_return"])

In [ ]:
# Drawdown Periods
qs.plots.drawdowns_periods(returns=vd["value_return"])

In [ ]:
# Drawdown
qs.plots.earnings(returns=vd["value_return"], start_balance=100000)

In [ ]:
# Heatmap
qs.plots.monthly_heatmap(returns=vd["value_return"])

In [ ]:
# Cumulative Returns
qs.plots.returns(returns=vd["value_return"], benchmark=vd["bm_return"])

In [ ]:
# Cumulative Returns
# qs.plots.yearly_returns(returns=vd["value_return"], benchmark=vd["bm_return"])

In [ ]:
# Rolling Sharpe
qs.plots.rolling_sharpe(
    returns=vd["value_return"],
    benchmark=vd["bm_return"],
    period=126,
    period_label="6-Months",
)

In [ ]:
# Rolling beta
qs.plots.rolling_beta(
    returns=vd["value_return"],
    benchmark=vd["bm_return"],
    window1=126,
    window1_label="6-Months",
    window2=252,
    window2_label="12-Months",
)

In [ ]:
# Rolling volatility
qs.plots.rolling_volatility(
    returns=vd["value_return"],
    benchmark=vd["bm_return"],
    period=126,
    period_label="6-Months",
)